# VITS Italian TTS Inference

This notebook demonstrates inference with a finetuned VITS model for Italian.
We'll look at how to:
- Run the base TTS post-finetuning
- Apply text tweaks (accents, apostrophes, padding)
- Prompt the model with a reference audio to mimic intonation

**Note:** The model checkpoint (`best_model.pth`) and config file (`config.json`) are downloaded automatically.

In [1]:
!pip install coqui-tts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 73.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 70.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 64.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.5/857.5 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from IPython.display import Audio
    

Uncomment the following lines to try your own examples:

In [30]:
import torch
from TTS.utils.synthesizer import Synthesizer
import soundfile as sf
import os


Using device: cuda
Audio saved to inference_output.wav


In [69]:
!gdown --id 1Uro2gsqQ8SrcWHwx4BK8FCrBu-Oc0rPt --output best_model.pth
!wget -O config.json "https://drive.google.com/file/d/1gKd85Q7-yBO8A-f-fxWvazoWkDnuChfs"

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1Uro2gsqQ8SrcWHwx4BK8FCrBu-Oc0rPt
From (redirected): https://drive.google.com/uc?id=1Uro2gsqQ8SrcWHwx4BK8FCrBu-Oc0rPt&confirm=t&uuid=82b1d775-3a8e-4293-b7b5-83ba34ef53c5
To: /content/best_model1.pth
100% 998M/998M [00:09<00:00, 109MB/s]
--2025-09-04 00:27:47--  https://drive.google.com/file/d/1gKd85Q7-yBO8A-f-fxWvazoWkDnuChfs
Resolving drive.google.com (drive.google.com)... 108.177.127.113, 108.177.127.101, 108.177.127.102, ...
Connecting to drive.google.com (drive.google.com)|108.177.127.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/file/d/1gKd85Q7-yBO8A-f-fxWvazoWkDnuChfs/ [following]
--2025-09-04 00:27:47--  https://drive.google.co

In [ ]:
model_checkpoint_path = "best_model.pth"
config_path = "config.json"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

synthesizer = Synthesizer(
    tts_checkpoint=model_checkpoint_path,
    tts_config_path=config_path,
    use_cuda=torch.cuda.is_available(),

)

In [62]:
wav = synthesizer.tts(text="Mercoledì ventiquattro luglio, alle undici ventiquattro ora locale, un violento terremoto ha scosso la penisola di Kamchatca, in Russia, provocando un allerta tsunami in tutto il Pacifico. ", language_name='it')
sf.write("0_base_inference_80k_steps.wav", wav, synthesizer.output_sample_rate)
Audio("0_base_inference_80k_steps.wav")

\+ generally good

\- need better padding at EOS

\- Kamchatka pronunciation

\- general stresses?

---

Trying to tweak the text prompt:
- accents, double letters
- Italian spelling
- lowercase

In [33]:
wav = synthesizer.tts(text="mercoledì ventiquattro luglio, alle undici ventiquattro ora locale, un violento terrem'oto ha scosso la pen'isola di camci'atca, in russia, provocando un all'erta tsun'ami in tutto il paciifico. ", language_name='it')
sf.write("1_accents_apostrophies_double_chars.wav", wav, synthesizer.output_sample_rate)
Audio("1_accents_apostrophies_double_chars.wav")

\+ sounds better

\- exaggerated stress on pacìfico

---

Trying helper padding at BOS

In [41]:
wav = synthesizer.tts(text=", mercoledì ventiquattro luglio, alle undici ventiquattro ora locale, un violento terremoto ha scosso la penisola di camciatca, in russia, provocando un allerta tsunami in tutto il pacifico. ", language_name='it')
sf.write("2_small_letters_padding_comma.wav", wav, synthesizer.output_sample_rate)
Audio("2_small_letters_padding_comma.wav")

\+ sounds really good

-> let's try adding styling prompt

---

Using a real example from sports news narration

In [53]:
Audio("upbeat_narration_sports_example.wav")

Using a sample in a basic case without text prompt preprocessing

In [63]:
wav = synthesizer.tts(
    text="Mercoledì ventiquattro luglio, alle undici ventiquattro ora locale, un violento terremoto ha scosso la penisola di Kamchatca, in Russia, provocando un allerta tsunami in tutto il Pacifico. ",
    language_name='it',
    speaker_wav='upbeat_narration_sports_example.wav'
    )
sf.write("3_prompting_with_voice_sample.wav", wav, synthesizer.output_sample_rate)
Audio("3_prompting_with_voice_sample.wav")

---

Tweaking stresses:

In [66]:
wav = synthesizer.tts(
    text="mercoledì ventiquattro luglio, alle undici ventiquattro ora locale, un violento terrem'oto ha scosso la pen'isola di camci'atca, in russia, provocando un all'erta tsun'ami in tutto il pacifico. ",
    language_name='it',
    speaker_wav='upbeat_narration_sports_example.wav'
    )
sf.write("4_prompting_with_voice_sample_and_stresses.wav", wav, synthesizer.output_sample_rate)
Audio("4_prompting_with_voice_sample_and_stresses.wav")

---
Tweaking stresses and leading comma

In [68]:
wav = synthesizer.tts(
    text=", mercoledì ventiquattro luglio, alle undici ventiquattro ora loc'ale, un violento terrem'oto ha scosso la peniisola di camci'atca, in russia, provocando un all'erta tsun'ami in tutto il pacifico. ",
    language_name='it',
    speaker_wav='upbeat_narration_sports_example.wav'
    )
sf.write("5_voice_sample_and_more_stresses.wav", wav, synthesizer.output_sample_rate)
Audio("5_voice_sample_and_more_stresses.wav")